In [1]:
import spacy
import pandas as pd

In [2]:
from lxml import etree
from datetime import datetime

In [3]:
def month_dates(start, end):
    f = lambda date: date.month + 12 * date.year

    res = []
    for tot_m in range(f(start)-1, f(end)):
        y, m = divmod(tot_m, 12)
        res.append(str(y) + '/' + '%02d' % (m+1))
    
    return res

In [4]:
def get_date(article):
    """
    This method returns the date of the article
    """
    str_date = article.find('entity').find('meta').find('issue_date').text
    return datetime.strptime(str_date, '%d/%m/%Y')

In [5]:
def get_articles_in_file(file, start_date, end_date):
    articles = []  
    for article in file.iter('article'):
        if article.find('entity') is not None:
            a = ''
            date = get_date(article)
            if start_date <= date <= end_date:
                for entity in article.iter('entity'):
                    a += entity.findtext('full_text') + ' '
                articles.append(date.strftime('%d/%m/%Y') + ' ' + a)
    return articles

In [6]:
def get_articles(path, start_date, end_date):
    articles = []
    for m_date in month_dates(start_date, end_date):
        try:
            file = etree.parse(path + m_date + '.xml')
            articles.append(get_articles_in_file(file, start_date, end_date))
        except (FileNotFoundError, IOError):
            pass
    return [a for file in articles for a in file]  

In [7]:
"""def get_articles(path, start_date, end_date):
    articles = []
    for m_date in month_dates(start_date, end_date):
        try:
            parser = etree.parse(path + m_date + '.xml')

            for article in parser.iter('article'):
                if article.find('entity') is not None:
                    a = ''
                    date = get_date(article)
                    if start_date <= date <= end_date:
                        for entity in article.iter('entity'):
                            a += entity.findtext('full_text') + ' '
                        articles.append(date.strftime('%d/%m/%Y') + ' ' + a)
                    
        except (FileNotFoundError, IOError):
            pass
    return articles"""

"def get_articles(path, start_date, end_date):\n    articles = []\n    for m_date in month_dates(start_date, end_date):\n        try:\n            parser = etree.parse(path + m_date + '.xml')\n\n            for article in parser.iter('article'):\n                if article.find('entity') is not None:\n                    a = ''\n                    date = get_date(article)\n                    if start_date <= date <= end_date:\n                        for entity in article.iter('entity'):\n                            a += entity.findtext('full_text') + ' '\n                        articles.append(date.strftime('%d/%m/%Y') + ' ' + a)\n                    \n        except (FileNotFoundError, IOError):\n            pass\n    return articles"

In [8]:
"""# retrieve the articles from JDL dataset
start = date(1826, 1, 1); end = date(1998, 2, 28)
articles = []
for date in month_dates(start, end):
    path = '/home/mbanga/Desktop/JDG/' + date + '.xml'
    try
        parser = etree.parse(path)
        tag_text = 'full_text'

        for article in parser.getroot().iterchildren():
            a = ''
            for elem in article.iterchildren():
                a += elem.findtext(tag_text) + ' '
            articles.append(a)

    except (FileNotFoundError, IOError):
        pass"""

"# retrieve the articles from JDL dataset\nstart = date(1826, 1, 1); end = date(1998, 2, 28)\narticles = []\nfor date in month_dates(start, end):\n    path = '/home/mbanga/Desktop/JDG/' + date + '.xml'\n    try\n        parser = etree.parse(path)\n        tag_text = 'full_text'\n\n        for article in parser.getroot().iterchildren():\n            a = ''\n            for elem in article.iterchildren():\n                a += elem.findtext(tag_text) + ' '\n            articles.append(a)\n\n    except (FileNotFoundError, IOError):\n        pass"

In [9]:
path = '/home/mbanga/Desktop/JDG/'
start_date =  datetime(1826, 1, 1)
end_date = datetime(1998, 2, 28)

In [10]:
articles = get_articles(path, start_date, end_date)

In [11]:
len(articles)

2997628

In [12]:
nlp = spacy.load('fr')

In [13]:
test_corpus = articles[-50:]

In [14]:
parsed_article = nlp(test_corpus[48])

In [15]:
for num, sentence in enumerate(parsed_article.sents):
    print('Sentence {}:' .format(num + 1))
    print(sentence)
    print('')

Sentence 1:
28/02/1998 Mœurs Pamela Anderson demande (encore) le divorce Pamela Anderson a demandé jeudi à divorcer de son mari, le batteur Tommy Lee. Ce dernier a été arrêté deux jours plus tôt à leur domicile pour avoir agressé l'actrice. Il s'agit de ta deuxième requête en divorce déposée par Pamela Anderson en deux ans. Le couple s'était réconcilié après la naissance de leur premier fils, Brandon, et Pamela Anderson avait dit qu'elle voulait aider son époux à vaincre son alcoolisme. Selon l'actrice, Tommy Lee l'a attaquée alors qu'elle ber-çait leur fils de sept semaines, Dylan Jagger. Elle a été légèrement blessée à la main et présente des marques rouges sur le dos. La police appelée à leur domicile de Malibu y a découvert une arme, en violation du sursis avec mise à l'épreuve auquel le batteur était soumis pour avoir agressé un photographe. (ATS / Reuter) Vin Attention aux mélanges... Le Tribunal de Sierre a confirmé vendredi la peine de trois mois de prison avec sursis contre un

In [16]:
token_pos = [token.pos_ for token in parsed_article]
token_text = [token.orth_ for token in parsed_article]
token_shape = [token.shape_ for token in parsed_article]
token_oov = [token.is_oov for token in parsed_article]

t = pd.Series(token_text, name='token_text')
p = pd.Series(token_pos, name='token_pos')
s = pd.Series(token_shape, name='token_shape')
oov = pd.Series(token_oov, name='token_oov')

df = pd.concat([t, p, s, oov], axis=1)

df.head()

,token_text,token_pos,token_shape,token_oov
0,28/02/1998,NUM,dd/dd/dddd,True
1,Mœurs,PROPN,Xxxxx,True
2,Pamela,PROPN,Xxxxx,True
3,Anderson,PROPN,Xxxxx,True
4,demande,VERB,xxxx,True


In [17]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [18]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuaiton or whitespace
    """
    
    return token.is_punct or token.is_space

In [19]:
def lemmatized_sentence_corpus(corpus):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    j = 0
    i = 0
    for parsed_article in nlp.pipe(corpus, 
                                   batch_size=50, n_threads=1):
        
        yield u' '.join([token.lemma_ for token in parsed_article
                             if not punct_space(token)])

In [20]:
#for lemma in lemmatized_sentence_corpus(test_corpus):
#    print(lemma)
#    print('\n')

In [35]:
articles[2990350]

"01/01/1998 Natel : Swisscom passe à l'offensive contre la protection des données L'opérateur national des télécommunications juge irréprochable son fichier sur les téléphones portables. Et accuse l'office chargé de la protection des données d'avoir cautionné un coup médiatique contre elle. L% entreprise Swisscom s'emploie depuis trois jours à désamorcer l' « afr faire des Natel ». Hier, l'opérateur unique (jusqu'au 31 décembre) des télécommunications en Suisse a défendu la légalité de son fichier sur les télé-phones portables, dont l'existence a été révélée dimanche. Swisscom accuse en outre l'office du préposé fédéral à la protection des données, Odilo Guntern, de « négligence » dans le traitement de l'affaire. Des représentants de Swisscom, de l'Office fédéral de la communication (OFCOM) et de la Police fédérale l'ont affirmé mardi : il n'y a pas de surveillance des mouvements des utilisateurs de Natel par Swisscom. Les conversations ne sont pas enregistrées et les autorités judicia